In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import os
from PIL import Image, ImageFile
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from google.colab import drive
from tqdm import tqdm

drive.mount('/content/drive', force_remount=True)

SAVE_DIR = '/content/drive/MyDrive/ProjectML-@LAS/ProjectML-@LAS/'
MODEL_PATH = os.path.join(SAVE_DIR, 'my_retrieval_model_final_atlas.h5')
BASE_DIR = os.path.join(SAVE_DIR, 'Google Images')

TRAIN_DIR = os.path.join(BASE_DIR, 'train')
VAL_DIR = os.path.join(BASE_DIR, 'val')
TEST_DIR = os.path.join(BASE_DIR, 'test', 'test_')

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 128
ImageFile.LOAD_TRUNCATED_IMAGES = True

model = load_model(MODEL_PATH, compile=False)

target_layer_name = None
for layer in model.layers:
    if "global_average_pooling" in layer.name:
        target_layer_name = layer.name
        break

if target_layer_name is None:
    raise ValueError("❌ ไม่พบเลเยอร์ global_average_pooling ในโมเดล")

inp = tf.keras.layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
x = inp
for layer in model.layers:
    x = layer(x)
    if layer.name == target_layer_name:
        break
feature_extractor = Model(inputs=inp, outputs=x)
print("✅ Feature extractor ready! Output shape:", feature_extractor.output_shape)

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)
label_map = {v: k for k, v in train_generator.class_indices.items()}

def build_feature_database(base_dirs, output_dir):
    all_features, all_paths = [], []

    for base_dir in base_dirs:
        print(f"\n📂 Processing folder: {base_dir}")
        for root, _, files in os.walk(base_dir):
            for file in tqdm(files, desc=os.path.basename(root)):
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    img_path = os.path.join(root, file)
                    try:
                        img = load_img(img_path, target_size=IMAGE_SIZE)
                        img_array = np.expand_dims(img_to_array(img), axis=0)
                        img_array = preprocess_input(img_array)
                        feature = feature_extractor.predict(img_array, verbose=0)
                        all_features.append(feature.flatten())
                        all_paths.append(img_path)
                    except Exception as e:
                        print(f"⚠️ Skip: {img_path} ({e})")

    np.save(os.path.join(output_dir, 'all_features.npy'), np.array(all_features))
    np.save(os.path.join(output_dir, 'all_paths.npy'), np.array(all_paths))
    print("\n✅ Features database rebuilt successfully!")
    print(f"Total: {len(all_paths)} images | Shape: {np.array(all_features).shape}")

# 🧩 ถ้าต้องการสร้างใหม่ ให้ uncomment
build_feature_database([TRAIN_DIR, VAL_DIR, TEST_DIR], SAVE_DIR)

all_features = np.load(os.path.join(SAVE_DIR, 'all_features.npy'))
all_paths = np.load(os.path.join(SAVE_DIR, 'all_paths.npy'), allow_pickle=True)
print("📦 Loaded feature DB:", all_features.shape)

def predict_and_find_similar(input_path, num_results=5, batch_size=128):
    img = load_img(input_path, target_size=IMAGE_SIZE)
    img_prepped = np.expand_dims(img_to_array(img), axis=0)
    img_prepped = preprocess_input(img_prepped)

    predictions_proba = model.predict(img_prepped, verbose=0)
    predicted_class_index = np.argmax(predictions_proba[0])
    predicted_label = label_map[predicted_class_index]
    confidence = np.max(predictions_proba[0])
    print(f"🔍 Predict: {predicted_label} | Confidence: {confidence * 100:.2f}%")

    all_classes_from_path = [os.path.basename(os.path.dirname(p)) for p in all_paths]
    filtered_indices = [i for i, c in enumerate(all_classes_from_path) if c == predicted_label]
    filtered_features = all_features[filtered_indices]
    filtered_paths = [str(all_paths[i]) for i in filtered_indices]

    query_feature_full = feature_extractor.predict(img_prepped, verbose=0).flatten()

    input_feature = query_feature_full[:filtered_features.shape[1]]

    distances = []
    num_batches = int(np.ceil(len(filtered_features) / batch_size))
    for i in tqdm(range(num_batches), desc="Distances (batches)"):
        start = i * batch_size
        end = min((i + 1) * batch_size, len(filtered_features))
        batch_features = filtered_features[start:end]
        batch_dist = np.linalg.norm(batch_features - input_feature, axis=1)
        distances.extend(batch_dist)
    distances = np.array(distances)

    similar_indices = np.argsort(distances)[:num_results]
    final_results = [(filtered_paths[i], distances[i]) for i in similar_indices]

    num_columns = num_results + 1
    plt.figure(figsize=(18, 6))

    plt.subplot(1, num_columns, 1)
    plt.imshow(img)
    plt.title(f"Query\n({predicted_label})")
    plt.axis('off')

    for i, (path, dist) in enumerate(final_results):
        plt.subplot(1, num_columns, i + 2)
        sim_img = Image.open(path)
        plt.imshow(sim_img)
        plt.title(f"Rank {i+1}\nDist: {dist:.4f}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()


MY_QUERY_IMAGE = '/content/drive/MyDrive/ProjectML-@LAS/ProjectML-@LAS/Google Images/test/test_/Pyramids Of Giza - Egypt/180.jpg'
# MY_QUERY_IMAGE = '/content/bg-mobile.jpg'
# MY_QUERY_IMAGE = '/content/kmutnb.png'

if os.path.exists(MY_QUERY_IMAGE):
    predict_and_find_similar(MY_QUERY_IMAGE, num_results=5, batch_size=128)
else:
    print(f"❌ ไม่พบไฟล์: {MY_QUERY_IMAGE}")
